In [63]:
import spacy, nltk, gensim, sklearn
import pandas as pd

In [64]:
year = 2020 # available: from 2015 to 2020
PATH_DATA = './data/data_nlp/'
QUOTES_FILE = PATH_DATA + f'quotes-{year}-filtered.json.bz2'
CHUNK_SIZE = 1000

reader = pd.read_json(QUOTES_FILE, lines=True, compression='bz2', chunksize=CHUNK_SIZE, typ='frame')

for chunk in reader:
    df_0 = chunk
    break
df_0

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,2020-04-15-000176,[ Queen ] can. He can cover.,None,[],2020-04-15 17:30:45,1,"[[None, 0.8956], [Eric DeCosta, 0.0665], [Kenn...",[https://www.pennlive.com/baltimore-ravens/202...,E
1,2020-02-03-000717,"50-YEAR-OLD J-LO, 43-YEAR-OLD SHAKIRA STAR IN ...",None,[],2020-02-03 16:51:49,1,"[[None, 0.6264], [Jennifer Lopez, 0.1782], [Je...",[http://sfgate.com/entertainment/article/Let-s...,E
2,2020-03-18-000741,A face-to-face duty lawyer service provided by...,Mike Dwyer,[Q6379626],2020-03-18 07:47:15,12,"[[Mike Dwyer, 0.6042], [None, 0.3958]]",[http://www.balonnebeacon.com.au/news/tourism-...,E
3,2020-01-20-001168,A lot of people think the Women's March is jus...,None,[],2020-01-20 04:00:00,1,"[[None, 0.7123], [Amanda Jones, 0.2877]]",[http://blogs.dallasobserver.com/news/dallas-w...,E
4,2020-01-16-001680,A lot of women say they remove some of my pain...,Mike Armour,[Q56425425],2020-01-16 12:11:32,1,"[[Mike Armour, 0.9256], [None, 0.0744]]",[https://www.abc.net.au/triplej/programs/hack/...,E
...,...,...,...,...,...,...,...,...,...
995,2020-03-10-042723,"More than the stylish movie it is, more than t...",Caryn James,[Q1046243],2020-03-10 18:11:00,1,"[[Caryn James, 0.9364], [None, 0.0636]]",[https://www.broadwayworld.com/st-louis/articl...,E
996,2020-01-27-051880,"Mother's Daugher (Wuki Remix),",Tracy Young,[Q7831883],2020-01-27 00:00:00,6,"[[Tracy Young, 0.7657], [None, 0.1384], [Chris...",[https://www.wuky.org/post/62nd-grammy-award-w...,E
997,2020-03-10-043036,Ms. Angelovic is relieved the order is over an...,John Barnett,"[Q16215177, Q21460325]",2020-03-10 15:00:31,1,"[[John Barnett, 0.8171], [None, 0.1829]]",[https://www.newuniversity.org/2020/03/10/murd...,E
998,2020-03-01-023724,"My daughter is eight, (Owen) is 11. Until that...",Paul Konerko,[Q685623],2020-03-01 23:15:54,2,"[[Paul Konerko, 0.8269], [None, 0.1731]]",[https://www.nbcsports.com/chicago/white-sox/p...,E


In [65]:
#Putting all the quotes in one corpus
text = ""

for quote in df_0.quotation:
    text = text + ' ' + quote

#Removing the new lines
text = " ".join(text.split())
text[:600]

"[ Queen ] can. He can cover. 50-YEAR-OLD J-LO, 43-YEAR-OLD SHAKIRA STAR IN HYPER-SEXUAL `HALF-TIME' , A face-to-face duty lawyer service provided by Legal Aid Queensland is NOT AVAILABLE at this time, A lot of people think the Women's March is just about feminism, but it stands for so much more, A lot of women say they remove some of my pain, but not all of it. Additionally, I oppose (bills) mandating schools to teach sex education beginning in kindergarten, as suggested in A6512. I believe this bill strips away the rights of parents to be involved in their children's sex education, and cause "

In [66]:
nlp = spacy.load('en_core_web_sm')
nlp.remove_pipe('parser')
nlp.remove_pipe('tagger')

#put in raw text, get a Spacy object
doc = nlp(text)
doc2 = nlp(text)

In [67]:
#loading default spacy stopword list
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS
len(spacy_stopwords)

#creating new stopword list
sw_list = list(sw[0]) + list(sw[1]) + list(sw[2]) + list(sw[3]) + list(sw[4]) + list(spacy_stopwords)
nlp.Defaults.stop_words = sw_list

print(len(nlp.Defaults.stop_words))


871


In [68]:
tokens = [token.text for token in doc]
len(tokens)

38166

In [69]:
PATH = PATH_DATA + 'stopword_list.csv'
sw = pd.read_csv(PATH, header=None)
sw

,0,1,2,3,4
0,a's,able,about,above,according
1,accordingly,across,actually,after,afterwards
2,again,against,ain't,all,allow
3,allows,almost,alone,along,already
4,also,although,always,am,among
...,...,...,...,...,...
104,why,will,willing,wish,with
105,within,without,won't,wonder,would
106,wouldn't,yes,yet,you,you'd
107,you'll,you're,you've,your,yours


In [70]:
#number of stopwords detected in text
stop_words = [token.text for token in doc if token.is_stop]
len(stop_words)


21244

In [71]:
words = [token.text for token in doc if token.is_stop != True and token.is_punct != True]
len(words)

12719

In [73]:
from collections import Counter
# five most common tokens
word_freq = Counter(words)
word_freq.most_common()
print(word_freq.most_common(100))

[('women', 118), ('people', 89), ('time', 70), ('woman', 53), ('family', 52), ('years', 48), ('life', 41), ('great', 41), ('good', 37), ('work', 37), ('wife', 37), ('day', 35), ('lot', 34), ('mother', 32), ('love', 32), ('children', 31), ('year', 31), ('girls', 31), ('world', 30), ('play', 30), ('team', 29), ('home', 28), ('girl', 28), ('dollars', 27), ('sexual', 27), ('young', 27), ('female', 27), ('daughter', 26), ('country', 25), ('person', 23), ('wanted', 23), ('hard', 23), ('big', 22), ('working', 21), ('beautiful', 20), ('thing', 20), ('support', 19), ('mom', 19), ('feel', 19), ('kind', 19), ('public', 19), ('married', 19), ('continue', 18), ('men', 18), ('case', 18), ('making', 17), ('marriage', 17), ('thought', 17), ('friends', 17), ('happy', 17), ('Women', 16), ('sex', 16), ('story', 16), ('things', 16), ('days', 16), ('loved', 16), ('community', 16), ('found', 16), ('told', 16), ('long', 15), ('strong', 15), ('hope', 15), ('fact', 15), ('close', 15), ('important', 15), ('baby